In [ ]:
import serial
import time
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

# Configuração da porta serial
port = '/dev/ttyACM0'  # Ajuste conforme necessário
baud_rate = 9600

# Inicializa a conexão serial
ser = serial.Serial(port, baud_rate)
time.sleep(2)  # Aguarda o Arduino estar pronto

# Arquivo CSV para salvar os dados
filename = "sensor_data.csv"

# Função para processar e salvar os dados
def salvar_dados():
    # Definir cabeçalho do CSV
    header = ["Timestamp", "Temperature", "Humidity", "Pressure", "AccX", "AccY", "AccZ", "GyroX", "GyroY", "GyroZ", "MagX", "MagY", "MagZ"]
    
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(header)  # Escrever cabeçalho

        # Lê e processa os dados do Arduino
        for i in tqdm(range(60), desc="Coletando Dados", unit="s"):
            line = ser.readline().decode('utf-8').strip()

            # Filtra linhas válidas
            if "Timestamp" in line:
                print(f"Recebido: {line}")  # Mostrar para depuração

                # Processa a linha e extrai os valores
                try:
                    timestamp = float(line.split(':')[1].strip().replace("s", "").strip())
                    temp = float(ser.readline().decode('utf-8').strip().split(":")[1].strip().replace("ºC", "").strip())
                    humidity = float(ser.readline().decode('utf-8').strip().split(":")[1].strip().replace("%", "").strip())
                    pressure = float(ser.readline().decode('utf-8').strip().split(":")[1].strip().replace("hPa", "").strip())
                    acc_x = float(ser.readline().decode('utf-8').strip().split(":")[1].strip().replace("m/s²", "").strip())
                    acc_y = float(ser.readline().decode('utf-8').strip().split(":")[1].strip().replace("m/s²", "").strip())
                    acc_z = float(ser.readline().decode('utf-8').strip().split(":")[1].strip().replace("m/s²", "").strip())
                    gyro_x = float(ser.readline().decode('utf-8').strip().split(":")[1].strip().replace("º/s", "").strip())
                    gyro_y = float(ser.readline().decode('utf-8').strip().split(":")[1].strip().replace("º/s", "").strip())
                    gyro_z = float(ser.readline().decode('utf-8').strip().split(":")[1].strip().replace("º/s", "").strip())
                    mag_x = float(ser.readline().decode('utf-8').strip().split(":")[1].strip().replace("µT", "").strip())
                    mag_y = float(ser.readline().decode('utf-8').strip().split(":")[1].strip().replace("µT", "").strip())
                    mag_z = float(ser.readline().decode('utf-8').strip().split(":")[1].strip().replace("µT", "").strip())

                    # Salva no arquivo CSV
                    writer.writerow([timestamp, temp, humidity, pressure, acc_x, acc_y, acc_z, gyro_x, gyro_y, gyro_z, mag_x, mag_y, mag_z])
                except Exception as e:
                    print(f"Erro ao processar linha: {line} | Erro: {e}")
            time.sleep(1)  # Aguarda 1 segundo antes de coletar novos dados

    print("Dados salvos no arquivo:", filename)

# Função para visualizar os dados
def visualizar_dados():
    # Carregar os dados do CSV
    data = pd.read_csv(filename)

    # Visualizar gráficos
    plt.figure(figsize=(10, 6))

    # Gráficos das grandezas
    plt.subplot(2, 2, 1)
    plt.plot(data['Timestamp'], data['Temperature'], label='Temperatura (ºC)')
    plt.xlabel('Tempo (s)')
    plt.ylabel('Temperatura (ºC)')
    plt.grid(True)

    plt.subplot(2, 2, 2)
    plt.plot(data['Timestamp'], data['Pressure'], label='Pressão (hPa)', color='r')
    plt.xlabel('Tempo (s)')
    plt.ylabel('Pressão (hPa)')
    plt.grid(True)

    plt.subplot(2, 2, 3)
    plt.plot(data['Timestamp'], data['AccX'], label='Aceleração X (m/s²)', color='g')
    plt.xlabel('Tempo (s)')
    plt.ylabel('Aceleração X (m/s²)')
    plt.grid(True)

    plt.subplot(2, 2, 4)
    plt.plot(data['Timestamp'], data['AccZ'], label='Aceleração Z (m/s²)', color='b')
    plt.xlabel('Tempo (s)')
    plt.ylabel('Aceleração Z (m/s²)')
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# Chama as funções
salvar_dados()  # Coleta e salva os dados
visualizar_dados()  # Visualiza os gráficos


Coletando Dados:   0%|          | 0/60 [00:00<?, ?s/s]